In [1]:
import os # built-in functions for working with the Windows system
import pandas as pd # most popular data science library for working with panel/tabular data
import numpy as np

In [2]:
# define path for users
username = os.getlogin()
if username == "root":
    username = os.getenv("USER")

if username == "rose775":
    path_2024_1 = "/Users/rose775/Library/CloudStorage/OneDrive-PNNL/General - NEB Decarb/Datasets/ResStock/2024.1"
    path_2024_2 = "/Users/rose775/Library/CloudStorage/OneDrive-PNNL/General - NEB Decarb/Datasets/ResStock/2024.2"
    path_out_2024_1 = "/Users/rose775/Library/CloudStorage/OneDrive-PNNL/General - NEB Decarb/Analysis/resstock_results_2/2024_1"
    path_out_2024_2 = "/Users/rose775/Library/CloudStorage/OneDrive-PNNL/General - NEB Decarb/Analysis/resstock_results_2/2024_2"

elif username == "kieren_username":
    path_2024_1 = "/General - NEB Decarb/Datasets/ResStock/2024.1"
    path_2024_2 = "/General - NEB Decarb/Datasets/ResStock/2024.2"
    path_out_2024_1 = "/General - NEB Decarb/Analysis/resstock_results_2/2024_1"
    path_out_2024_2 = "/General - NEB Decarb/Analysis/resstock_results_2/2024_2"

elif username == "max_username":
    path_2024_1 = "/General - NEB Decarb/Datasets/ResStock/2024.1"
    path_2024_2 = "/General - NEB Decarb/Datasets/ResStock/2024.2"
    path_out_2024_1 = "/General - NEB Decarb/Analysis/resstock_results_2/2024_1"
    path_out_2024_2 = "/General - NEB Decarb/Analysis/resstock_results_2/2024_2"

else:
    print("Who are you and why are you doing ResStock analysis?")

In [3]:
files_2024_1 = ["baseline_metadata_and_annual_results.parquet", "upgrade1.04_metadata_and_annual_results.parquet", "upgrade2.01_metadata_and_annual_results.parquet",
                "upgrade2.02_metadata_and_annual_results.parquet", "upgrade2.03_metadata_and_annual_results.parquet", "upgrade2.04_metadata_and_annual_results.parquet",
                "upgrade2.05_metadata_and_annual_results.parquet", "upgrade3.03_metadata_and_annual_results.parquet", "upgrade3.07_metadata_and_annual_results.parquet",
                "upgrade4.04_metadata_and_annual_results.parquet"]

files_2024_2 = ["baseline_metadata_and_annual_results.parquet", "upgrade01_metadata_and_annual_results.parquet", "upgrade02_metadata_and_annual_results.parquet",
                "upgrade03_metadata_and_annual_results.parquet", "upgrade04_metadata_and_annual_results.parquet", "upgrade05_metadata_and_annual_results.parquet",
                "upgrade06_metadata_and_annual_results.parquet", "upgrade07_metadata_and_annual_results.parquet", "upgrade08_metadata_and_annual_results.parquet",
                "upgrade09_metadata_and_annual_results.parquet", "upgrade10_metadata_and_annual_results.parquet", "upgrade14_metadata_and_annual_results.parquet",
                "upgrade15_metadata_and_annual_results.parquet"]

In [4]:
def read_resstock_files(files, path_in):
    dfs = {} # create empty dictionary to fill with Pandas dataframes

    # Loop through each file name, read the file, and store the df in dfs dictionary
    if files is files_2024_1: # make sure we"re reading files from 2024v1
        for file in files:
            x = file.split("_")[0] # just use the "baseline" or "upgradeXX" information for naming the dfs
            df_name = f"{x}_2024_1"
            dfs[df_name] = pd.read_parquet(f"{path_in}/{file}").reset_index() # read the parquet and save the named df to the dict
    elif files is files_2024_2:
        for file in files:
            x = file.split("_")[0]
            df_name = f"{x}_2024_2"
            dfs[df_name] = pd.read_parquet(f"{path_in}/{file}").reset_index()

    keys = list(dfs.keys())

    return dfs, keys

In [ ]:
def df_diff(df, baseline):
    # Step 1: Identify numeric columns in both DataFrames
    numeric_cols_df = df.select_dtypes(include="number").columns
    numeric_cols_baseline = baseline.select_dtypes(include="number").columns

    # Exclude "weight" and "in.sqft" from the numeric columns
    cols_to_exclude = ["weight", "in.sqft"]
    numeric_cols_df = numeric_cols_df.difference(cols_to_exclude)
    numeric_cols_baseline = numeric_cols_baseline.difference(cols_to_exclude)

    # Step 2: Subtract matching numeric columns excluding "weight" and "in.sqft"
    diff_df = df.copy()  # Make a copy of df to store the differences
    for col in numeric_cols_df.intersection(numeric_cols_baseline):
        diff_df[col] -= baseline[col]

    # Step 3: Add "weight" and "in.sqft" back to the DataFrame
    diff_df[cols_to_exclude] = df[cols_to_exclude]

    # Step 4: Create a new DataFrame with the differences including "weight" and "in.sqft"
    diff_df = diff_df.assign(**{col: diff_df[col] for col in numeric_cols_df})

    return diff_df

In [ ]:
def resstock_summary(df_dict, path_out):
    baseline_key = [key for key in df_dict.keys() if "baseline" in key][0]
    keys = [key for key in df_dict.keys() if key != baseline_key]
    baseline = df_dict[baseline_key]
    for key in keys:
        # resstock measure
        df = df_dict[key]
        
        # get difference between baseline and measure dfs (df_measure - df_baseline)
        diff_df = df_diff(df, baseline)

        # calculate the weighted values by multiplying row["weight"] by row[numerical_column]
        df_weighted = diff_df.copy()
        numerical_cols = df_weighted.select_dtypes(include=np.number).drop(columns=["weight", "in.sqft"]).columns
        df_weighted[numerical_cols] = df_weighted[numerical_cols].mul(df_weighted["weight"], axis=0)

        # aggregate weighted means and unweighted sums
        conditions = ["out.emissions_reduction.all_fuels", "in.sqft", "bldg_id", "out.site_energy.net.energy_consumption.kwh"]
        numerical_cols = df_weighted.select_dtypes(include=np.number).columns
        agg_dict_1 = {col: "mean" for col in numerical_cols if any(condition in col.lower() for condition in conditions)}
        agg_dict_1["bldg_id"] = "count"
        agg_dict_1["in.county"] = "nunique"
        agg_dict_2 = {col: "sum" for col in numerical_cols if any(condition in col.lower() for condition in conditions)}

        unweighted_mean = diff_df.groupby("in.ashrae_iecc_climate_zone_2004").agg(agg_dict_1).reset_index()
        weighted_sum = df_weighted.groupby("in.ashrae_iecc_climate_zone_2004").agg(agg_dict_2).reset_index()

        # merge for output
        result = pd.merge(unweighted_mean, weighted_sum, on = "in.ashrae_iecc_climate_zone_2004", suffixes = ("_avg_site", "_entire_cz"))
        
        # to .csv
        result.to_csv(f"{path_out}/{key}_results.csv")


### Carbon analysis

In [ ]:
# first do the 2024v1 files
dfs_2024_1 = read_resstock_files(files_2024_1, path_2024_1)[0]
resstock_summary(dfs_2024_1, path_out_2024_1)

In [ ]:
# delete the 2024v1 dfs to clear up some RAM
del dfs_2024_1

In [ ]:
# now do the 2024v2 files
dfs_2024_2 = read_resstock_files(files_2024_2, path_2024_2)[0]
resstock_summary(dfs_2024_2, path_out_2024_2)

### Population table

In [5]:
dfs_2024_1 = read_resstock_files(files_2024_1, path_2024_1)[0]


In [6]:
dfs_2024_1["baseline_2024_1"].groupby("in.ashrae_iecc_climate_zone_2004").agg({"in.county": "nunique",
                                                                                  "weight": "sum"
                                                                                  }).round(0).to_clipboard()

In [ ]:
del dfs_2024_1

In [ ]:
dfs_2024_2 = read_resstock_files(files_2024_2, path_2024_2)[0]

In [ ]:
dfs_2024_2["baseline_2024_2"].groupby("in.ashrae_iecc_climate_zone_2004").agg({"in.county": "nunique",
                                                                                  "weight": "sum"}).round(0).to_clipboard()